# The code will implement the training pipeline for the CNN regressor mentioned in https://www.nature.com/articles/s41598-021-02387-9

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## IMPORT

In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import matplotlib.pyplot as plt
import json
import codecs
from tqdm import tqdm
import glob


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("We're using:", device)

We're using: cuda


In [4]:
cd /content/drive/Shareddrives/CIS519_Spring2023/labeled_data

/content/drive/Shareddrives/CIS519_Spring2023/labeled_data


In [5]:
label_path = "/content/drive/Shareddrives/CIS519_Spring2023/labeled_data/labeled_input/metadata/metadata/train_sequence_counts.csv"
df_label = pd.read_csv(label_path)
df_label.head()

,seq_id,count
0,95dae922-21bc-11ea-a13a-137349068a90,9
1,2fff6c8a-7d42-11eb-8fb5-0242ac1c0002,9
2,300cc650-7d42-11eb-8fb5-0242ac1c0002,9
3,30125958-7d42-11eb-8fb5-0242ac1c0002,9
4,301d2d24-7d42-11eb-8fb5-0242ac1c0002,9


In [6]:
# simplify annotation frame
f = open('labeled_input/metadata/metadata/iwildcam2022_train_annotations.json')
with codecs.open("labeled_input/metadata/metadata/iwildcam2022_train_annotations.json", 'r',
                 encoding='utf-8', errors='ignore') as f:
    train_meta = json.load(f)
df_seq = pd.DataFrame(train_meta['images'])
df_seq = df_seq.drop(["seq_num_frames","datetime","width","width","height","id","sub_location","location"],axis=1)
df_seq['count_per_image'] = [0]*len(df_seq)
df_seq = df_seq.where(df_seq['seq_id'].isin(list(df_label['seq_id']))).dropna()
df_seq = df_seq.reset_index(drop=True)
df_seq.head()

,seq_id,file_name,seq_frame_num,count_per_image
0,5c8f4d8a-7ed0-11eb-b7d6-0242ac1c0002,8ab0ba0e-21bc-11ea-a13a-137349068a90.jpg,0.0,0.0
1,5c8f4d8a-7ed0-11eb-b7d6-0242ac1c0002,9159fece-21bc-11ea-a13a-137349068a90.jpg,1.0,0.0
2,5c8f4d8a-7ed0-11eb-b7d6-0242ac1c0002,9807316a-21bc-11ea-a13a-137349068a90.jpg,2.0,0.0
3,5c8f4d8a-7ed0-11eb-b7d6-0242ac1c0002,98ebf30e-21bc-11ea-a13a-137349068a90.jpg,3.0,0.0
4,5c8f4d8a-7ed0-11eb-b7d6-0242ac1c0002,989a0b20-21bc-11ea-a13a-137349068a90.jpg,4.0,0.0


In [ ]:
for i in tqdm(range(len(df_label))):
  seq_id = df_label.iloc[i]['seq_id']
  count = df_label.iloc[i]['count']
  df_seq['count_per_image'] = np.where(df_seq['seq_id']==seq_id, count, df_seq['count_per_image'])

100%|██████████| 1780/1780 [00:02<00:00, 658.87it/s]


In [ ]:
# get rid of files that doesn't exist in image file
new_df_seq = df_seq
for i in tqdm(range(len(df_seq))):
  img_name = os.path.join("labeled_input/train/train", df_seq.iloc[i]['file_name'])
  if not os.path.isfile(img_name):
    new_df_seq = new_df_seq.drop(i, inplace=False)

new_df_seq = new_df_seq.reset_index(drop=True)

100%|██████████| 13190/13190 [00:02<00:00, 6261.59it/s]


In [ ]:
new_df_seq.head(5)

,seq_id,file_name,seq_frame_num,count_per_image
0,5c8f4d8a-7ed0-11eb-b7d6-0242ac1c0002,8ab0ba0e-21bc-11ea-a13a-137349068a90.jpg,0.0,2.0
1,5c8f4d8a-7ed0-11eb-b7d6-0242ac1c0002,9159fece-21bc-11ea-a13a-137349068a90.jpg,1.0,2.0
2,5c8f4d8a-7ed0-11eb-b7d6-0242ac1c0002,9807316a-21bc-11ea-a13a-137349068a90.jpg,2.0,2.0
3,5c8f4d8a-7ed0-11eb-b7d6-0242ac1c0002,98ebf30e-21bc-11ea-a13a-137349068a90.jpg,3.0,2.0
4,5c8f4d8a-7ed0-11eb-b7d6-0242ac1c0002,989a0b20-21bc-11ea-a13a-137349068a90.jpg,4.0,2.0


## CNN Architecture

  ### Load pretrained model
  

In [ ]:
import torch.nn as nn
import torchvision.models as models

# Load a pre-trained ResNet-18 model
resnet = models.resnet101(pretrained=True)

# Freeze all layers except the last two
for param in resnet.parameters():
    param.requires_grad = False
for param in resnet.fc.parameters():
    param.requires_grad = True

# Replace the last fully connected layer with a regression layer
resnet.fc = nn.Linear(resnet.fc.in_features, 1)
resnet = resnet.to(device)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 229MB/s]


In [ ]:
#!mv resnet152b.pth trained_models/resnet152b.pth

## Inference

In [ ]:
PATH = "/content/drive/Shareddrives/CIS519_Spring2023/resnet101b.pth"
resnet.load_state_dict(torch.load(PATH))
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Test set:

In [7]:
f_path = "/content/drive/Shareddrives/CIS519_Spring2023/labeled_data/labeled_input/metadata/metadata/iwildcam2022_train_annotations.json"
f = open(f_path)
with codecs.open(f_path , 'r',
                 encoding='utf-8', errors='ignore') as f:
    train_meta = json.load(f)
df_seq = pd.DataFrame(train_meta['images'])
df_seq = df_seq.drop(["seq_num_frames","datetime","width","width","height","id","sub_location","location"],axis=1)
df_seq.head()

,seq_id,file_name,seq_frame_num
0,30048d32-7d42-11eb-8fb5-0242ac1c0002,8b02698a-21bc-11ea-a13a-137349068a90.jpg,0
1,30048d32-7d42-11eb-8fb5-0242ac1c0002,8e5b81de-21bc-11ea-a13a-137349068a90.jpg,1
2,30048d32-7d42-11eb-8fb5-0242ac1c0002,8c6be0e4-21bc-11ea-a13a-137349068a90.jpg,2
3,30048d32-7d42-11eb-8fb5-0242ac1c0002,8fdf7998-21bc-11ea-a13a-137349068a90.jpg,3
4,30048d32-7d42-11eb-8fb5-0242ac1c0002,96093c50-21bc-11ea-a13a-137349068a90.jpg,4


In [8]:
with codecs.open("/content/drive/Shareddrives/CIS519_Spring2023/labeled_data/labeled_input/metadata/metadata/iwildcam2022_mdv4_detections.json", 'r',
                 encoding='utf-8', errors='ignore') as f:
    md_meta = json.load(f)

In [9]:
seq_counts = pd.read_csv('/content/drive/Shareddrives/CIS519_Spring2023/labeled_data/labeled_input/metadata/metadata/train_sequence_counts.csv')  
seq_counts.head()

,seq_id,count
0,95dae922-21bc-11ea-a13a-137349068a90,9
1,2fff6c8a-7d42-11eb-8fb5-0242ac1c0002,9
2,300cc650-7d42-11eb-8fb5-0242ac1c0002,9
3,30125958-7d42-11eb-8fb5-0242ac1c0002,9
4,301d2d24-7d42-11eb-8fb5-0242ac1c0002,9


In [10]:
def count_useful_detections(row, threshold):
  return len([x for x in row if (x['category']=='1' and x['conf']>threshold)])

In [11]:
conf_thresh = 0.75

#: based on the various metadata structures, get a useful training frame from Mega Detector
df_seq = df_seq.where(df_seq['seq_id'].isin(list(seq_counts['seq_id']))).dropna()
df_seq["file_name"]= 'train/' + df_seq['file_name'].astype(str)
df_mega = pd.DataFrame(md_meta['images'])
df_mega_full_train = df_mega.where(df_mega['file'].isin(list(df_seq['file_name']))).dropna()
vfunc = np.vectorize(count_useful_detections)
df_mega_full_train['detections'] = vfunc(df_mega_full_train['detections'], conf_thresh)
_df_seq = df_seq.rename({'file_name': 'file'}, axis=1).drop('seq_frame_num',axis=1)
df_mega_full_train = pd.merge(df_mega_full_train, _df_seq, on='file', how='inner')
df_mega_full_train

<ipython-input-11-3747dea739e7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seq["file_name"]= 'train/' + df_seq['file_name'].astype(str)


,file,max_detection_conf,detections,seq_id
0,train/8dae8cae-21bc-11ea-a13a-137349068a90.jpg,0.974,1,301cceec-7d42-11eb-8fb5-0242ac1c0002
1,train/922368fe-21bc-11ea-a13a-137349068a90.jpg,0.999,3,300a44e8-7d42-11eb-8fb5-0242ac1c0002
2,train/994d9d8e-21bc-11ea-a13a-137349068a90.jpg,0.754,1,302b1902-7d42-11eb-8fb5-0242ac1c0002
3,train/9936f6e2-21bc-11ea-a13a-137349068a90.jpg,0.998,1,9936f868-21bc-11ea-a13a-137349068a90
4,train/91d35990-21bc-11ea-a13a-137349068a90.jpg,0.955,1,300a242c-7d42-11eb-8fb5-0242ac1c0002
...,...,...,...,...
13185,train/9067a962-21bc-11ea-a13a-137349068a90.jpg,0.997,1,30138dfa-7d42-11eb-8fb5-0242ac1c0002
13186,train/8bc84b32-21bc-11ea-a13a-137349068a90.jpg,0.999,4,30270e34-7d42-11eb-8fb5-0242ac1c0002
13187,train/8a5f5790-21bc-11ea-a13a-137349068a90.jpg,0.997,1,979882d8-21bc-11ea-a13a-137349068a90
13188,train/8f113920-21bc-11ea-a13a-137349068a90.jpg,0.999,3,3012584a-7d42-11eb-8fb5-0242ac1c0002


## get ground truth

## RESNET

In [ ]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
test_results = pd.DataFrame(columns =['Id','Predicted'])
test_results = pd.DataFrame({'Id': pd.Series(dtype='string'),
                   'Predicted': pd.Series(dtype='int')})

test_df_seq = df_mega_full_train[0:1000]

for idx, row  in test_df_seq.iterrows():
    if idx % 10 == 0:
        print(idx," images done")
    img_name = os.path.join("/content/drive/Shareddrives/CIS519_Spring2023/labeled_data/labeled_input/train", row['file'])
    seq = row['seq_id']
    if os.path.isfile(img_name):
      im = Image.open(img_name)
      im = np.array(im)
      imnpy = torch.from_numpy(im)
      imnpy = imnpy.permute(2, 0, 1)
      imnpy = imnpy.to(torch.float32)
      test_image = imnpy.unsqueeze(0)
      test_image = test_image.to(device)
    
      infer = resnet(test_image)
      y_pred = min(max(0,round(infer.cpu().detach().numpy().flatten()[0])),10)
      existing = test_results[test_results['Id']==seq]
      if len(existing) == 0:
          test_results = pd.concat([test_results,pd.DataFrame({'Id': [seq],'Predicted': [y_pred]})], ignore_index=True)
      else:
          test_results.loc[test_results['Id']==seq, 'Predicted'] = max(y_pred,int(existing['Predicted']))


0  images done
10  images done
20  images done
30  images done
40  images done
50  images done
60  images done
70  images done
80  images done
90  images done
100  images done
110  images done
120  images done
130  images done
140  images done
150  images done
160  images done
170  images done
180  images done
190  images done
200  images done
210  images done
220  images done
230  images done
240  images done
250  images done
260  images done
270  images done
280  images done
290  images done
300  images done
310  images done
320  images done
330  images done
340  images done
350  images done
360  images done
370  images done
380  images done
390  images done
400  images done
410  images done
420  images done
430  images done
440  images done
450  images done
460  images done
470  images done
480  images done
490  images done
500  images done
510  images done
520  images done
530  images done
540  images done
550  images done
560  images done
570  images done
580  images done
590  ima

In [ ]:
test_results

,Id,Predicted
0,301cceec-7d42-11eb-8fb5-0242ac1c0002,1
1,300a44e8-7d42-11eb-8fb5-0242ac1c0002,2
2,302b1902-7d42-11eb-8fb5-0242ac1c0002,1
3,9936f868-21bc-11ea-a13a-137349068a90,2
4,300a242c-7d42-11eb-8fb5-0242ac1c0002,0
...,...,...
755,3021fcdc-7d42-11eb-8fb5-0242ac1c0002,2
756,2fffd65c-7d42-11eb-8fb5-0242ac1c0002,1
757,302bb614-7d42-11eb-8fb5-0242ac1c0002,3
758,300fdf20-7d42-11eb-8fb5-0242ac1c0002,1


In [ ]:
test_results['Predicted'].unique()

array([1, 2, 0, 3, 4])

In [ ]:
test_results.to_csv("/content/drive/Shareddrives/CIS519_Spring2023/train_data_prediction/resnet101b_predictions.csv", sep=',', index=False)

In [ ]:
df_count = pd.read_csv("/content/drive/Shareddrives/CIS519_Spring2023/labeled_data/labeled_input/metadata/metadata/train_sequence_counts.csv")

In [ ]:
df_count

,seq_id,count
0,95dae922-21bc-11ea-a13a-137349068a90,9
1,2fff6c8a-7d42-11eb-8fb5-0242ac1c0002,9
2,300cc650-7d42-11eb-8fb5-0242ac1c0002,9
3,30125958-7d42-11eb-8fb5-0242ac1c0002,9
4,301d2d24-7d42-11eb-8fb5-0242ac1c0002,9
...,...,...
1775,9925c372-21bc-11ea-a13a-137349068a90,8
1776,9932df3a-21bc-11ea-a13a-137349068a90,8
1777,99556aa0-21bc-11ea-a13a-137349068a90,5
1778,99577458-21bc-11ea-a13a-137349068a90,4


In [ ]:
labels = np.zeros(len(test_results))
for i in range(len(test_results)):
  seq_id = test_results.iloc[i]['Id']
  labels[i] = df_count[df_count['seq_id'] == seq_id]["count"]

In [ ]:
np.save('/content/drive/Shareddrives/CIS519_Spring2023/train_data_prediction/labels.npy', labels)

### Megadetector Inference

In [21]:
df_mega_test = pd.DataFrame(columns =['Id','Predicted'])
df_mega_test = pd.DataFrame({'Id': pd.Series(dtype='string'),
                   'Predicted': pd.Series(dtype='int')})
test_results = pd.read_csv("/content/drive/Shareddrives/CIS519_Spring2023/train_data_prediction/resnet101b_predictions.csv")


In [22]:
test_results

,Id,Predicted
0,301cceec-7d42-11eb-8fb5-0242ac1c0002,1
1,300a44e8-7d42-11eb-8fb5-0242ac1c0002,2
2,302b1902-7d42-11eb-8fb5-0242ac1c0002,1
3,9936f868-21bc-11ea-a13a-137349068a90,2
4,300a242c-7d42-11eb-8fb5-0242ac1c0002,0
...,...,...
755,3021fcdc-7d42-11eb-8fb5-0242ac1c0002,2
756,2fffd65c-7d42-11eb-8fb5-0242ac1c0002,1
757,302bb614-7d42-11eb-8fb5-0242ac1c0002,3
758,300fdf20-7d42-11eb-8fb5-0242ac1c0002,1


In [23]:
for i in range(len(test_results)):
  seq = test_results.iloc[i]['Id']
  y_pred = max(df_mega_full_train[df_mega_full_train['seq_id'] == seq]["detections"])
  df_mega_test = pd.concat([df_mega_test,pd.DataFrame({'Id': [seq],'Predicted': [y_pred]})], ignore_index=True)
  
  

In [27]:
df_mega_test

,Id,Predicted
0,301cceec-7d42-11eb-8fb5-0242ac1c0002,3
1,300a44e8-7d42-11eb-8fb5-0242ac1c0002,4
2,302b1902-7d42-11eb-8fb5-0242ac1c0002,3
3,9936f868-21bc-11ea-a13a-137349068a90,2
4,300a242c-7d42-11eb-8fb5-0242ac1c0002,1
...,...,...
755,3021fcdc-7d42-11eb-8fb5-0242ac1c0002,4
756,2fffd65c-7d42-11eb-8fb5-0242ac1c0002,3
757,302bb614-7d42-11eb-8fb5-0242ac1c0002,4
758,300fdf20-7d42-11eb-8fb5-0242ac1c0002,3


In [28]:
df_mega_test.to_csv("/content/drive/Shareddrives/CIS519_Spring2023/train_data_prediction/megadetector_predictions.csv", sep=',', index=False)